# Tests de performance application IPS-rest et IPS-graphql

## Préparation : génération de la topologie et fonctions de génération d'intents

Fonctions de création de la topologie avec les APIs Rest et GraphQL :

In [1]:
import requests
from requests.auth import HTTPBasicAuth
import networkx as nx
import matplotlib.pyplot as plt
import time
import itertools
import random
import numpy as np

gq_req_size = 0
rest_req_size = 0
ip = "192.168.1.154"

def create_graph_graphql(ip):
    global gq_req_size
    # initialisation du graphe
    gr = nx.DiGraph()

    # requêtes au contrôleur ONOS
    url = "http://"+ip+":5000/graphql"
    liens_query = {'query': "query{liens{src,dst}}"}
    host_query = {'query': "query{hosts{id,locations}}"}
    r_hosts = requests.post(url, data = host_query)
    r_links = requests.post(url, data = liens_query)
    gq_req_size = len(r_hosts.content + r_links.content)

    if (r_hosts.status_code != 200):
        return "Erreur sur la liste des hôtes."
    elif (r_links.status_code != 200):
        return "Erreur sur la liste des liens."
    else:
        global host_list_mac_gq
        host_list_mac_gq = []
        host_list = r_hosts.json()['data']['hosts']
        link_list = r_links.json()['data']['liens']
        
        for l in link_list:
            gr.add_edge(l['src']['device'],l['dst']['device'],orig=l['src']['port'],dest=l['dst']['port'])
            gr.add_edge(l['dst']['device'],l['src']['device'],orig=l['dst']['port'],dest=l['src']['port'])
        
        for h in host_list:
            gr.add_node(h['id'],id=h['id'])
            host_list_mac_gq.append(h['id'])
            gr.add_edge(h['id'],h['locations'][0]["elementId"],orig="host",dest=h['locations'][0]["port"])
            gr.add_edge(h['locations'][0]["elementId"],h['id'],orig=h['locations'][0]["port"],dest="host")
        
        return gr
    
def create_graph_rest(ip):
    global rest_req_size
    # initialisation du graphe
    gr = nx.DiGraph()

    # requêtes au contrôleur ONOS
    r_host = requests.get("http://"+ip+":5000/hosts")
    r_link = requests.get("http://"+ip+":5000/links")
    rest_req_size = len(r_host.content + r_link.content)
    if (r_host.status_code != 200):
        return "Erreur sur la liste des hôtes."
    elif (r_link.status_code != 200):
        return "Erreur sur la liste des liens."
    else:
        global host_list_mac_re
        host_list_mac_re = []
        host_list = r_host.json()
        link_list = r_link.json()
        
        for l in link_list:
            gr.add_edge(l['src']['device'],l['dst']['device'],orig=l['src']['port'],dest=l['dst']['port'])
            gr.add_edge(l['dst']['device'],l['src']['device'],orig=l['dst']['port'],dest=l['src']['port'])
        
        for h in host_list:
            gr.add_node(h['id'],id=h['id'])
            host_list_mac_re.append(h['id'])
            gr.add_edge(h['id'],h['locations'][0]["elementId"],orig="host",dest=h['locations'][0]["port"])
            gr.add_edge(h['locations'][0]["elementId"],h['id'],orig=h['locations'][0]["port"],dest="host")
        
        return gr

Fonctions de génération d'intents

In [2]:
def install_intent_re(gr_rest,mac_o,mac_d,liste_chemin,count):
    l=liste_chemin
    fwd_orig = ("","")
    fwd_dest = ("","")
    mac_orig = mac_o.replace("/None","")
    mac_dest = mac_d.replace("/None","")
    for i in range(0,len(l)-1):
        if i==len(l)-2:
            port = gr_rest.get_edge_data(l[i],l[i+1])
            fwd_dest = (l[i],port["orig"])
            intent_orig = fwd_orig[0]+"-"+fwd_orig[1]
            intent_dest = fwd_dest[0]+"-"+fwd_dest[1]
            
            r_intent = requests.get("http://"+ip+":5000/intent?orig="+intent_orig+"&dest="+intent_dest
            +"&macorig="+mac_orig+"&macdest="+mac_dest)
            r_intent_inv = requests.get("http://"+ip+":5000/intent?orig="+intent_dest+"&dest="+intent_orig
            +"&macorig="+mac_dest+"&macdest="+mac_orig)
            count += 2
        else:
            port = gr_rest.get_edge_data(l[i],l[i+1])
            if (port["orig"]=="host"):
                fwd_orig = (l[i+1],port["dest"])
            else:
                fwd_dest = (l[i],port["orig"])
                if (port["orig"]!="host"):
                    intent_orig = fwd_orig[0]+"-"+fwd_orig[1]
                    intent_dest = fwd_dest[0]+"-"+fwd_dest[1]
                    
                    r_intent = requests.get("http://"+ip+":5000/intent?orig="+intent_orig+"&dest="+intent_dest
                    +"&macorig="+mac_orig+"&macdest="+mac_dest)
                    r_intent_inv = requests.get("http://"+ip+":5000/intent?orig="+intent_dest+"&dest="+intent_orig
                    +"&macorig="+mac_dest+"&macdest="+mac_orig)
                    count += 2

                    fwd_orig = (l[i+1],port["dest"])
                else:
                    fwd_orig = ("","")
    return count

def install_intent_gq(gr_graphql,mac_o,mac_d,liste_chemin,count):
    l=liste_chemin
    fwd_orig = ("","")
    fwd_dest = ("","")
    mac_orig = mac_o.replace("/None","")
    mac_dest = mac_d.replace("/None","")
    string_mut="mutation { \n"
    
    for i in range(0,len(l)-1):
        if i==len(l)-2:
            port = gr_graphql.get_edge_data(l[i],l[i+1])
            fwd_dest = (l[i],port["orig"])
            intent_orig = fwd_orig[0]+"/"+fwd_orig[1]
            intent_dest = fwd_dest[0]+"/"+fwd_dest[1]
            string_mut += "r"+str(i)+": addIntent(intent_orig: \""+intent_orig+"\", mac_orig: \""+mac_orig+"\", intent_dest: \""+intent_dest+"\", mac_dest: \""+mac_dest+"\") { intent_orig, intent_dest }\n"
            string_mut += "s"+str(i)+": addIntent(intent_orig: \""+intent_dest+"\", mac_orig: \""+mac_dest+"\", intent_dest: \""+intent_orig+"\", mac_dest: \""+mac_orig+"\") { intent_orig, intent_dest }\n"
        else:
            port = gr_graphql.get_edge_data(l[i],l[i+1])
            if (port["orig"]=="host"):
                fwd_orig = (l[i+1],port["dest"])
            else:
                fwd_dest = (l[i],port["orig"])
                if (port["orig"]!="host"):
                    intent_orig = fwd_orig[0]+"/"+fwd_orig[1]
                    intent_dest = fwd_dest[0]+"/"+fwd_dest[1]

                    string_mut += "r"+str(i)+": addIntent(intent_orig: \""+intent_orig+"\", mac_orig: \""+mac_orig+"\", intent_dest: \""+intent_dest+"\", mac_dest: \""+mac_dest+"\") { intent_orig, intent_dest }\n"
                    string_mut += "s"+str(i)+": addIntent(intent_orig: \""+intent_dest+"\", mac_orig: \""+mac_dest+"\", intent_dest: \""+intent_orig+"\", mac_dest: \""+mac_orig+"\") { intent_orig, intent_dest }\n"

                    fwd_orig = (l[i+1],port["dest"])
                else:
                    fwd_orig = ("","")

    string_mut += "}"
    mutation_query = {'query': string_mut}
    query_gql = requests.post("http://"+ip+":5000/graphql", data = mutation_query)
    return count+1

def liste_couples_res():
    liste_couples = list(itertools.combinations(host_list_mac_re, 2))
    liste_res = []

    global ent
    ent = np.floor(len(liste_couples)/10).astype(int)
    for i in range(1,11):
        liste_res.append(random.choices(liste_couples, k=ent*i))
    return liste_res

def do_install_intent(gr_rest,gr_graphql):
    temps_re = []
    temps_gq = []

    count_re = []
    count_gq = []

    liste_couples = liste_couples_res()

    for i in liste_couples:
        count = 0
        start = time.time()
        for elt in i:
            # clean = requests.get("http://"+ip+":5000/removeIntents")
            count = install_intent_re(gr_rest,elt[0],elt[1],nx.shortest_path(gr_rest,elt[0],elt[1]),count)
        end = time.time()
        temps_re.append(round(end-start,1))
        count_re.append(count)

    for i in liste_couples:
        count = 0
        start = time.time()
        for elt in i:
            # clean = requests.get("http://"+ip+":5000/removeIntents")
            count = install_intent_gq(gr_graphql,elt[0],elt[1],nx.shortest_path(gr_graphql,elt[0],elt[1]),count)
        end = time.time()
        temps_gq.append(round(end-start,1))
        count_gq.append(count)

    return [temps_re,temps_gq,count_re,count_gq]

def trace_resultats(l):
    pourc = []
    pourc2 = []
    for i in range(0,10):
        pourc.append((l[0][i]-l[1][i])*100/l[0][i])
        pourc2.append((l[2][i]-l[3][i])*100/l[2][i])
    print(pourc)
    print(np.mean(pourc))
    print(pourc2)
    print(np.mean(pourc2))

    labels = []
    for i in range(1,11):
        labels.append(str(ent*i))

    x = np.arange(len(labels))
    width = 0.42

    fig, ax = plt.subplots(figsize=(17,8))
    fig2, ax2 = plt.subplots(figsize=(17,8))

    rects1 = ax.bar(x - width/2, l[0], width, label='Rest')
    rects2 = ax.bar(x + width/2, l[1], width, label='GraphQL')

    rects3 = ax2.bar(x - width/2, l[2], width, label='Rest')
    rects4 = ax2.bar(x + width/2, l[3], width, label='GraphQL')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_xlabel('Nombre de couples générés')
    ax.set_ylabel('Temps en secondes')
    ax.set_title('Temps de génération des intents')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()

    ax.bar_label(rects1)
    ax.bar_label(rects2)

    fig.tight_layout()

    ax2.set_xlabel('Nombre de couples générés')
    ax2.set_ylabel('Nombre de requêtes')
    ax2.set_title('Nombre de requêtes nécessaires à la génération des intents')
    ax2.set_xticks(x)
    ax2.set_xticklabels(labels)
    ax2.legend()

    ax2.bar_label(rects3)
    ax2.bar_label(rects4)

    fig2.tight_layout()

    plt.rcParams["font.size"] = "20"
    plt.show()

## Tests : méthode

- on compte le nombre d'hôtes N de la topologie : le nombre de paires d'hôtes à relier au total est alors T = N*(N-1)/2
- on calcule une liste L de 10 nombres Ti régulièrement espacés de paires d'hôtes à relier, qui seront choisis au hasard parmi la liste de toutes les paires d'hôtes de la topologie
- pour chaque liste de paires d'hôtes à relier, on installe les intents nécessaires pour router correctement tous les paquets entre chaque paire d'hôte

### T1 : topologie torus,3,3 : 9 hôtes, 36 liens

In [3]:
gr_rest = create_graph_rest(ip)
gr_graphql = create_graph_graphql(ip)

l = do_install_intent(gr_rest,gr_graphql)
print(l)

[[0.0, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.3, 0.3, 0.3], [1.5, 2.6, 3.3, 4.7, 5.2, 6.9, 8.2, 9.6, 10.5, 12.2], [14, 30, 40, 58, 68, 92, 108, 124, 138, 154], [3, 6, 9, 12, 15, 18, 21, 24, 27, 30]]


In [4]:
trace_resultats(l)

ZeroDivisionError: float division by zero

### T2 : topologie torus,4,4 : 16 hôtes, 64 liens

In [ ]:
gr_rest = create_graph_rest(ip)
gr_graphql = create_graph_graphql(ip)

l = do_install_intent(gr_rest,gr_graphql)
print(l)

[[1.2, 2.4, 3.4, 4.6, 5.8, 7.3, 8.1, 9.6, 10.6, 11.7], [1.1, 2.4, 3.6, 4.5, 5.6, 7.2, 8.0, 9.1, 9.9, 11.4], [16, 32, 46, 60, 74, 94, 104, 122, 130, 148], [3, 6, 9, 12, 15, 18, 21, 24, 27, 30]]


In [ ]:
trace_resultats(l)